# Subset selection for Hyperparameter tuning

In this tutorial, we will look at an example showing how to integrate GradMatchPB-Warm subset selection strategy in typical hyperparameter tuning loop
for configuration model training on CIFAR100 dataset with TPE as hyper-parameter search algorithm and ASHA as hyper-parameter scheduler.

### Cloning CORDS repository

In [1]:
!git clone https://github.com/decile-team/cords.git
%cd cords/
%ls

Cloning into 'cords'...
remote: Enumerating objects: 3996, done.
remote: Counting objects: 100% (2618/2618), done.
remote: Compressing objects: 100% (1189/1189), done.
remote: Total 3996 (delta 1706), reused 2243 (delta 1390), pack-reused 1378
Receiving objects: 100% (3996/3996), 54.64 MiB | 17.93 MiB/s, done.
Resolving deltas: 100% (2443/2443), done.
/content/cords
benchmarks/  docs/        README.md      tests/        train_ssl.py
configs/     examples/    requirements/  train_hpo.py
cords/       LICENSE.txt  setup.py       train_sl.py


### Install prerequisite libraries of CORDS

In [ ]:
!pip install dotmap
!pip install apricot-select
!pip install ray[default]
!pip install ray[tune]
!pip install datasets

     |████████████████████████████████| 154 kB 7.4 MB/s 
  Created wheel for apricot-select: filename=apricot_select-0.6.1-py3-none-any.whl size=48786 sha256=1b4eb91dc07c416e6b5c124c69520332e5671ceec59601fa554b165ee465c127
  Stored in directory: /root/.cache/pip/wheels/1d/b0/5d/41bab30f23d17864700963dad70bbeda159a409e94f0778f2f
Successfully built apricot-select


### Import necessary libraries

In [ ]:
import argparse
from cords.utils.config_utils import load_config_data
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.skopt import SkOptSearch
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.suggest.ax import AxSearch
from ray.tune.suggest.bohb import TuneBOHB
from ray.tune.suggest.nevergrad import NevergradSearch
from ray.tune.suggest.optuna import OptunaSearch
from ray.tune.suggest.zoopt import ZOOptSearch
from ray.tune.suggest.sigopt import SigOptSearch
from ray.tune.suggest.hebo import HEBOSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.schedulers import HyperBandScheduler
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray import tune

### Loading hyperparameter configuration file with predefined arguments:

We have a set of predefined configuration files added to CORDS for HPO under cords/configs/HPO/ which can be used directly by loading them as a dotmap object. 

An example of predefined configuration for Hyper-parameter tuning on CIFAR100 dataset with ASHA as scheduler and TPE as search algorithm can be found below:

```Python
from ray import tune

config = dict(setting= "hyperparamtuning",

# parameter for subset selection
# all settings for subset selection will be fetched from here
subset_config = "configs/SL/config_gradmatchpb-warm_cifar100.py",

# parameters for hyper-parameter tuning
# search space for hyper-parameter tuning
space = dict(
        learning_rate=tune.uniform(0.001, 0.01), 
        learning_rate1=tune.uniform(0.001, 0.01),
        learning_rate2=tune.uniform(0.001, 0.01),
        learning_rate3=tune.uniform(0.001, 0.01),
        scheduler= tune.choice(['cosine_annealing', 'linear_decay']),
        nesterov= tune.choice([True, False]),
        gamma= tune.uniform(0.05, 0.5),    
        ),

# tuning algorithm 
search_algo = "TPE",

# number of hyper-parameter set to try
num_evals = 27,

# metric to be optimized, for 'mean_loss' metric mode should be 'min'
metric = "mean_accuracy",
mode = "max",

# scheduler to be used (i.e ASHAScheduler)
# scheduler terminates trials that perform poorly
# learn more here: https://docs.ray.io/en/releases-0.7.1/tune-schedulers.html
scheduler = 'hyperband',

# where to store logs
log_dir = "RayLogs/",

# resume hyper-parameter tuning from previous log
# specify 'name' (i.e main_2021-03-09_18-33-56) below
resume = False,

# only required if you want to resume from previous checkpoint
# it can also be specified if you don't want to resume
name = None,

# specify resources to be used per trial
# i.e {'gpu':1, 'cpu':2}
resources = {'gpu':1},

# if True, trains model on Full dataset with the best parameter selected.
final_train = True
)
```

Please find a detailed documentation explaining the available configuration parameters in the following readthedocs [page]()

***Loading the predefined configuration file directly using the load_config_data function in CORDS***

In [ ]:
from cords.utils.config_utils import load_config_data
param_tuning_cfg = load_config_data('/content/cords/configs/HPO/config_hyper-param_tuning_cifar100.py')

### Modifying Default SL training loop to include search space parameters as configurable parameters
In this example, the search space parameters are layer-wise learning rates, step-size decay, learning rate scheduler, and nesterov momentum boolean indicator. We modify the default pytorch training loop given in train_sl.py to include the hyper-parameter search space parameters in the following manner:

```python
def optimizer_with_scheduler(self, model):
        if self.cfg.optimizer.type == 'sgd':
            optimizer = optim.SGD( [
                                    {"params": model.linear.parameters(), "lr": self.cfg.optimizer.lr1},
                                    {"params": model.layer4.parameters(), "lr": self.cfg.optimizer.lr2},
                                    {"params": model.layer3.parameters(), "lr": self.cfg.optimizer.lr2},
                                    {"params": model.layer2.parameters(), "lr": self.cfg.optimizer.lr2},
                                    {"params": model.layer1.parameters(), "lr": self.cfg.optimizer.lr2},
                                    {"params": model.conv1.parameters(), "lr": self.cfg.optimizer.lr3},
                                    ],
                                    lr=self.cfg.optimizer.lr,
                                  momentum=self.cfg.optimizer.momentum,
                                  weight_decay=self.cfg.optimizer.weight_decay,
                                  nesterov=self.cfg.optimizer.nesterov)
        elif self.cfg.optimizer.type == "adam":
            optimizer = optim.Adam(model.parameters(), lr=self.cfg.optimizer.lr)
        elif self.cfg.optimizer.type == "rmsprop":
            optimizer = optim.RMSprop(model.parameters(), lr=self.cfg.optimizer.lr)

        if self.cfg.scheduler.type == 'cosine_annealing':
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                                   T_max=self.cfg.scheduler.T_max)
        elif self.cfg.scheduler.type == 'linear_decay':
            scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                                        step_size=self.cfg.scheduler.stepsize, 
                                                        gamma=self.cfg.scheduler.gamma)
        else:
            scheduler = None
        return optimizer, scheduler
```

We include the modified default train_sl loop in examples\HPO\image_classification\python_code\vision_train_sl.py file.

In [ ]:
import sys
sys.path.append('/content/cords/examples/HPO/image_classification/python_code/vision_train_sl')
from vision_train_sl import TrainClassifier

# Instantiating GradMatchPB-Warm arguments required for train_sl.py by loading the corresponding configuration file.

In [ ]:
train_cfg_file = '/content/cords/configs/SL/config_gradmatchpb-warm_cifar100.py'
train_cfg = load_config_data(train_cfg_file)

"""
Note that we have to do following changes to standard training configuration files
to get them working for Hyper-parameter tuning.
"""
train_cfg.report_tune = True
train_cfg.train_args.print_every = 1

#Instantiating the train classifier class with the loaded train_cfg
train_class = TrainClassifier(train_cfg)

### Get Hyper-parameter search algorithm 

In this example, we will be using Tree-structured parzen estimator(TPE) as the hyper-parameter search algorithm. In the hyper-parameter tuning configuration file, the search algorithm option is given as cfg.search_algo.

In [ ]:
method = param_tuning_cfg.search_algo
#Search space
space = param_tuning_cfg.space
#Evaluation metric for configuration evaluation
metric = param_tuning_cfg.metric
#maximum or minimum mode
mode = param_tuning_cfg.mode

"""
Shows all hyper-parameter search algorithm that work with CORDS. We use ray-tune library for hyper-parameter tuning
in CORDS. Hence, all search algorithms given in raytune can be used with CORDS as well.
"""
# HyperOptSearch 
if method == "hyperopt" or method == "TPE":
    search = HyperOptSearch(space, metric, mode)
# BayesOptSearch
elif method == "bayesopt" or method == "BO":
    search = BayesOptSearch(space, metric = metric, mode = mode)
# SkoptSearch
elif method == "skopt" or method == "SKBO":
    search = SkOptSearch(space, metric = metric, mode = mode)
# DragonflySearch
elif method == "dragonfly" or method == "SBO":
    search = DragonflySearch(space, metric = metric, mode = mode)
# AxSearch
elif method == "ax" or method == "BBO":
    search = AxSearch(space, metric = metric, mode = mode)
# TuneBOHB
elif method == "tunebohb" or method == "BOHB":
    search = TuneBOHB(space, metric = metric, mode = mode)
# NevergradSearch
elif method == "nevergrad" or method == "GFO":
    search = NevergradSearch(space, metric = metric, mode = mode)
# OptunaSearch
elif method == "optuna" or method == "OSA":
    search = OptunaSearch(space, metric = metric, mode = mode)
# ZOOptSearch
elif method == "zoopt" or method == "ZOO":
    search = ZOOptSearch(space, metric = metric, mode = mode)
# SigOptSearch
elif method == "sigopt":
    search = SigOptSearch(space, metric = metric, mode = mode)
# HEBOSearch
elif method == "hebo" or method == "HEBO":
    search = HEBOSearch(space, metric = metric, mode = mode)
else:
    search = None


### Get Hyper-parameter scheduler

In this example, we will be using ASHA as the hyper-parameter scheduler algorithm. In the hyper-parameter tuning configuration file, the scheduler option is given as cfg.scheduler.

In [ ]:
method = param_tuning_cfg.scheduler

if method == "ASHA" or method == "asha":
    scheduler = AsyncHyperBandScheduler(metric = metric, mode = mode, 
                                        max_t = train_cfg.train_args.num_epochs)
elif method == "hyperband" or method == "HB":
    scheduler = HyperBandScheduler(metric = metric, mode = mode, 
                max_t = train_cfg.train_args.num_epochs)
elif method == "BOHB":
    scheduler = HyperBandForBOHB(metric = metric, mode = mode)
else:
    scheduler = None

scheduler = scheduler

### Utility function that updates the training configuration parameters with new parameters suggested by search algorithm

In [ ]:
def update_parameters(config, new_config):
    # a generic function to update parameters
    if 'learning_rate' in new_config:
        config.optimizer.lr = new_config['learning_rate']
    if 'learning_rate1' in new_config:
        config.optimizer.lr1 = new_config['learning_rate1']
    if 'learning_rate2' in new_config:
        config.optimizer.lr2 = new_config['learning_rate2']
    if 'learning_rate3' in new_config:
        config.optimizer.lr3 = new_config['learning_rate3']
    if 'optimizer' in new_config:
        config.optimizer.type = new_config['optimizer']
    if 'nesterov' in new_config:
        config.optimizer.nesterov = new_config['nesterov']
    if 'scheduler' in new_config:
        config.scheduler.type = new_config['scheduler']
    if 'gamma' in new_config:
        config.scheduler.gamma = new_config['gamma']
    if 'epochs' in new_config:
        config.train_args.num_epochs = new_config['epochs']
    if 'trn_batch_size' in new_config:
        config.dataloader.batch_size = new_config['trn_batch_size']
    if 'hidden_size' in new_config:
        config.model.hidden_size = new_config['hidden_size']
    if 'num_layers' in new_config:
        config.model.num_layers = new_config['num_layers']
    return config


### Utility function that takes in the search configuration parameters suggested by hyper-parameter search algorithm, update the training configuration file accordingly, and train the model with the new configuration. 

In [ ]:
def param_tune(config):
    #update parameters in config dict
    new_config = update_parameters(train_cfg, config)
    train_cfg = new_config
    # turn on reporting to ray every time
    train_cfg.report_tune = True
    train_class.train()


### Start Hyper-parameter tuning

In [ ]:
analysis = tune.run(
          param_tune,
          num_samples=param_tuning_cfg.num_evals,
          search_alg=search,
          scheduler=scheduler,
          resources_per_trial=param_tuning_cfg.cfg.resources,
          local_dir=param_tuning_cfg.log_dir+train_cfg.dss_args.type+'/',
          log_to_file=True,
          name=param_tuning_cfg.name,
          resume=param_tuning_cfg.resume)

### Get best hyper-parameter configuration

In [ ]:
best_config = analysis.get_best_config(metric=param_tuning_cfg.metric, mode=param_tuning_cfg.mode)
print("Best Config: ", best_config)